### Проект “Scraps and losses report”


Вопрос: компания-производитель продуктов питания осуществляет продажу товаров в торговые объекты. В процессе возникает ситуация, когда торговые объекты возвращают обратно продукт компании (например, у продукта закончился срок годности, брак и т.п.). Это отражается на net sales со знаком минус

Цель: контроль и анализ уровня возвратов, оптимизация уровня возвратов

Задачи: подготовка отчета с таблицами и графиками об уровне возвратов

Инструменты: pandas, postgres, power bi

#### Этапы:

Сбор исходных данных  
Преобразование данных, построение таблиц  
Создание базы данных  
Обновление данных в БД  
Создание dashboard в Power BI  

#### Исходные данные:

Данные по продажам (sales)  
Данные по возвратам (returns)  
Данные по SKU  
Данные по клиентам (customers)  

#### Преобразование данных:

Работа в Jupyter Notebook с использованием библиотеки Pandas. Применение агрегирующих функций  
Подготовка dataframes и сsv файлов для загрузки в БД  
Создание сводных таблиц в разрезе brand, sub brand, clients и sales people name  


In [1]:
import pandas as pd
import numpy as np
import os
import datetime

In [2]:
lst = [os.path.join('export_returns', i) for i in os.listdir('export_returns')]
lst1 = sorted(lst, key = os.path.getmtime, reverse=True)
lst1_ = list(item for item in lst1 if "/." not in item)
lst1_

['export_returns/export_Jan_2023.xlsx',
 'export_returns/export_Oct_2023.xlsx',
 'export_returns/export_Sep_2023.xlsx',
 'export_returns/export_Aug_2023.xlsx',
 'export_returns/export_Jul_2023.xlsx',
 'export_returns/export_Jun_2023.xlsx',
 'export_returns/export_May_2023.xlsx',
 'export_returns/export_Apr_2023.xlsx',
 'export_returns/export_Mar_2023.xlsx',
 'export_returns/export_Feb_2023.xlsx']

In [3]:
a = lst1_[1]
a

'export_returns/export_Oct_2023.xlsx'

In [4]:
df_r = pd.read_excel(a)

#df_r.info()

In [5]:
df_r = df_r.rename(columns=lambda c: c.lower().replace('.', '_').replace(' ', '_'))

In [6]:
df_r_1 = df_r.loc[((df_r['sales_document_type'] == 'ZREV') \
                & (df_r['order_reason'] == 'ZU2') \
                & (df_r['reason_for_rejection'].isna()) \
                & (df_r['delivery_block'].isna()) \
                & (df_r['usage'] == 165))].reset_index(drop=True)

df_r_1['week_no'] = df_r_1['requested_deliv_date'].dt.isocalendar().week

df_r_1 = df_r_1.groupby(["ship_to", "material", "week_no"])\
                .agg({"order_quantity": "sum", "order_caf": "sum"})\
                .reset_index()

df_r_2 = df_r[df_r['sales_document_type'].isin(['ZRB'])\
            & df_r['order_reason'].isin(['ZUR'])\
            & df_r['reason_for_rejection'].isnull()\
            & df_r['delivery_block'].isnull()\
            & df_r['usage'].isin([165])].reset_index(drop=True)

df_r_2['week_no'] = df_r_2['requested_deliv_date'].dt.isocalendar().week

df_r_2 = df_r_2.groupby(['ship_to', 'material', 'week_no'])[['order_quantity', 'order_caf']].sum().reset_index()

df_r_3 = df_r.query("sales_document_type == 'ZRB' and order_reason == 'ZUR'\
                        and reason_for_rejection == 'B6' and delivery_block == 'Z3'\
                        and usage == 165").reset_index(drop=True)

df_r_3['week_no'] = df_r_3['requested_deliv_date'].dt.isocalendar().week

df_r_3 = df_r_3.groupby(['ship_to', 'material', 'week_no'])[['order_quantity', 'order_caf']].apply(np.sum, axis=0).reset_index()

In [7]:
df_r_123 = pd.concat([df_r_1, df_r_2, df_r_3], axis=0).reset_index(drop=True)

df_r_123['order_caf'] = df_r_123['order_caf'] * (-1)

df_returns = pd.pivot_table(df_r_123,
                            index = ['ship_to', 'material', 'week_no'],
                            values = ['order_quantity', 'order_caf'],
                            aggfunc = 'sum').reset_index()

df_returns = df_returns[['ship_to', 'material', 'week_no', 'order_quantity', 'order_caf']]

df_returns.rename(columns = {'order_quantity' : 'scraps_quantity', 'order_caf' : 'scraps_byn'}, inplace=True)

df_returns = df_returns.astype({'ship_to': 'int64', 'material' : 'int64', 'scraps_quantity' : 'int64'})

#df_returns.info()

In [8]:
df_returns_db = df_returns.copy()

df_returns_db['month'] = df_r.loc[1, 'requested_deliv_date'].month_name()

df_returns_db['year'] = df_r.loc[1, 'requested_deliv_date'].year

In [9]:
mn = df_r.loc[1, 'requested_deliv_date'].month_name()
mn

y = df_r.loc[1, 'requested_deliv_date'].year
y

period = f"{mn} {y}"
period

'October 2023'

In [10]:
first = df_r['requested_deliv_date'].min()
fd = first.strftime('%d.%m.%Y')
fd

second = df_r['requested_deliv_date'].max()
sd = second.strftime('%d.%m.%Y')
sd

dd = f"{fd}-{sd}"
dd

'01.10.2023-31.10.2023'

In [11]:
file_name_returns = 'returns_database/returns {}.csv'
file_name_returns = file_name_returns.format(dd)

#df_returns_db.to_csv(file_name_returns, index=False)
#print('File {} is created'.format(file_name_returns))

--

In [12]:
lstlst = [os.path.join('export_sales', i) for i in os.listdir('export_sales')]
lst2 = sorted(lstlst, key = os.path.getmtime, reverse=True)
lst2_ = list(item for item in lst2 if "/." not in item)
lst2_

['export_sales/export_Oct_2023.xlsx',
 'export_sales/export_Sep_2023.xlsx',
 'export_sales/export_Aug_2023.xlsx',
 'export_sales/export_Jul_2023.xlsx',
 'export_sales/export_Jun_2023.xlsx',
 'export_sales/export_May_2023.xlsx',
 'export_sales/export_Apr_2023.xlsx',
 'export_sales/export_Mar_2023.xlsx',
 'export_sales/export_Feb_2023.xlsx',
 'export_sales/export_Jan_2023.xlsx']

In [13]:
b = lst2_[0]
b

'export_sales/export_Oct_2023.xlsx'

In [14]:
df_sales = pd.read_excel(b)

In [15]:
df_sales.rename(columns = {'new Код ТТ' : 'ship_to', 'SAP code NEW' : 'material', 'Неделя' : 'week_no',\
                       'Отгружено шт' : 'sales_quantity', 'Отгружено CAF без НДС' : 'sales_byn'}, inplace=True)

In [16]:
lambda_fun = lambda x: x[-2 :]

df_sales['week_no'] = df_sales.loc[:, 'week_no'].apply(lambda_fun)

df_sales = df_sales.astype({'week_no' : 'UInt32'})

#df_sales.info()

In [17]:
df_sales_db = df_sales.copy()

df_sales_db['month'] = df_r.loc[1, 'requested_deliv_date'].month_name()

df_sales_db['year'] = df_r.loc[1, 'requested_deliv_date'].year

In [18]:
file_name_sales = 'sales_database/sales {}.csv'
file_name_sales = file_name_sales.format(dd)

#df_sales_db.to_csv(file_name_sales, index=False)
#print('File {} is created'.format(file_name_sales))

--

In [19]:
df_sales_returns = pd.merge(df_sales, df_returns, how = 'outer', on = ['ship_to', 'material', 'week_no'])

df_sales_returns = df_sales_returns.drop(columns = ['week_no'])

df_sales_returns = df_sales_returns.groupby(['ship_to', 'material']).sum().reset_index()

df_sales_returns = df_sales_returns.astype({'sales_quantity' : 'int64', 'scraps_quantity' : 'int64'})

#df_sales_returns.info()

--

In [20]:
df_sku = pd.read_excel('sku/sku.xlsx')

In [21]:
df_sku = df_sku.rename(columns=lambda c: c.lower().replace('.', '_').replace(' ', '_'))

#df_sku.info()

In [22]:
df_sku.rename(columns = {'product_id' : 'material', 'subbrand' : 'sub_brand', 'product_desc' : 'sku_name'}, inplace=True)

df_sku = df_sku.astype({'material' : 'int64'})

#df_sku.info()

In [23]:
#df_sku.to_csv('sku/sku.csv', index=False)

--

In [24]:
df_customers = pd.read_excel('customers/customers.xlsx')

#df_customers.info()

In [25]:
df_customers.rename(columns = {'(CUS) Ship To Code' : 'ship_to', '(CUS-DC) Proxi Category 2' : 'client',\
                               '(CUS-DC) Proxi Category 1' : 'cluster',\
                                '(CUS-SF4) Sales Representative' : 'sales_people_name',\
                                '(CUS-DC) Territory Level 2' : 'region'}, inplace=True)

#df_customers.info()

In [26]:
#df_customers.to_csv('customers/customers.csv', index=False)

--

In [27]:
df_sr_sku = pd.merge(df_sales_returns, df_sku, how = 'left', on = ['material'])

In [28]:
list_sku_nan = list(df_sr_sku['material'][df_sr_sku['sku_name'].isnull()].value_counts())
list_sku_nan

[]

In [29]:
if not list_sku_nan:
    print('there is all information about the sku')
else:
    raise UserWarning('UPDATE LIST OF SKU')

there is all information about the sku


In [30]:
df_brands = pd.pivot_table(df_sr_sku, index = ['brand'],
                        values = ['sales_quantity', 'sales_byn', 'scraps_quantity', 'scraps_byn'],
                        aggfunc = 'sum', margins=True)

In [31]:
df_brands = df_brands.astype({'sales_byn': 'int64', 'scraps_byn' : 'int64'})

df_brands['%_scraps_byn'] = df_brands['scraps_byn'] / df_brands['sales_byn']

df_brands['%_scraps_quantity'] = df_brands['scraps_quantity'] / df_brands['sales_quantity']

In [32]:
def color_(value):

  if 0 <= value < 0.03:
    color = 'black'
  elif 0.03 <= value <= 1:
    color = 'red'
  else:
    color = 'grey'

  return 'color: %s' % color

In [33]:
def highlight_last(x):
    
    return ['font-weight: bold' if v == x.iloc[-1] else '' for v in x]

In [34]:
df_brands_ = df_brands.style.map(color_, subset=['%_scraps_byn','%_scraps_quantity']).\
    format({'%_scraps_byn' : "{:.1%}", '%_scraps_quantity' : "{:.1%}"}, thousands=' ', decimal=',').\
    apply(highlight_last).\
    set_properties(subset=['sales_byn', 'scraps_byn', '%_scraps_byn'], **{'width': '150px'}).\
    set_caption(f"Scraps and losses in {period} by brands")

In [35]:
df_brands_

,sales_byn,sales_quantity,scraps_byn,scraps_quantity,%_scraps_byn,%_scraps_quantity
brand,,,,,,
AB,17 428 496,13 246 480,613 349,457 300,"3,5%","3,5%"
ACM,6 104 944,5 833 240,179,160,"0,0%","0,0%"
AM,22 232,15 870,135 212,120 660,"608,2%","760,3%"
AP,2 468 539,711 150,4 703,1 340,"0,2%","0,2%"
AV,14,20,15 558,12 220,"111128,6%","61100,0%"
DM,8 411 722,8 093 610,68 817,60 340,"0,8%","0,7%"
DN,15,30,653,1 210,"4353,3%","4033,3%"
PR,9 885 873,6 904 410,205 741,114 350,"2,1%","1,7%"
PT,3 474 421,403 110,23 677,2 710,"0,7%","0,7%"


Уровни возвратов выше 100% у таких брендов, как AV, AM и DN обусловлены тем, что данные бренды выведены из продажи и сейчас происходит списание. Это нормально, однако вывод бренда DN был осуществлен еще весной 2023, поэтому необходимо выяснить, почему до сих пор есть возвраты. Вывод бренда AV был в июле 2023, поэтому данный уровень списания (в абсолютных значениях) в принципе нормален. Вывод бренда AM осуществлен (осуществляется) в октябре 2023, поэтому высокие абсолютные значения возвратов тоже нормальны.  

Бренд AB пришел на замену бренда AV. Высокий уровень возвратов может говорить о том, что фактический спрос оказался ниже прогнозируемого. А также стоит проверить, понимают ли торговые представители, что заказывать стоит меньше.  

Бренд ACM пришел на замену бренда AM. Продажи только начались, возвраты еще не проводились.


---

In [36]:
df_sub_brands = pd.pivot_table(df_sr_sku, index = ['sub_brand'],
                        values = ['sales_quantity', 'sales_byn', 'scraps_quantity', 'scraps_byn'],
                        aggfunc = 'sum', margins=True)

df_sub_brands['%_scraps_byn'] = df_sub_brands['scraps_byn'] / df_sub_brands['sales_byn']

df_sub_brands['%_scraps_quantity'] = df_sub_brands['scraps_quantity'] / df_sub_brands['sales_quantity']

df_sub_brands = df_sub_brands.astype({'sales_byn': 'int64', 'scraps_byn' : 'int64'})

df_sub_brands = df_sub_brands.dropna()

#df_sub_brands

In [37]:
df_sub_brands.style.\
    map(color_, subset=['%_scraps_byn','%_scraps_quantity']).\
    format({'%_scraps_byn' : "{:.1%}", '%_scraps_quantity' : "{:.1%}"}, thousands=' ', decimal=',').\
    apply(highlight_last).\
    set_properties(subset=['sales_byn', 'scraps_byn', '%_scraps_byn'], **{'width': '150px'}).\
    set_caption(f"Scraps and losses in {period} by sub brands")

,sales_byn,sales_quantity,scraps_byn,scraps_quantity,%_scraps_byn,%_scraps_quantity
sub_brand,,,,,,
AB 1110,899 390,1 045 810,49 125,58 440,"5,5%","5,6%"
AB 130,2 877 274,2 968 640,53 392,55 400,"1,9%","1,9%"
AB 130 N,1 653 410,1 483 600,59 165,52 420,"3,6%","3,5%"
AB 140,3 132 233,2 519 750,82 455,65 760,"2,6%","2,6%"
AB 220,457 737,286 940,17 166,10 820,"3,8%","3,8%"
AB 260 N,8 408 450,4 941 740,352 043,214 460,"4,2%","4,3%"
ACM 1,5 991 738,5 812 940,179,160,"0,0%","0,0%"
ACM 6,113 205,20 300,0,0,"0,0%","0,0%"
AM 1 95,11 843,13 530,124 285,118 240,"1049,4%","873,9%"


In [38]:
df_sub_brands_1 = df_sub_brands.copy()

In [39]:
df_sub_brands_2 = (df_sub_brands_1[(df_sub_brands_1['%_scraps_byn'] > 0.025) & (df_sub_brands_1['%_scraps_byn'] <= 1)]).\
                    sort_values('%_scraps_byn', ascending=False)
#df_sub_brands_2

In [40]:
df_sub_brands_2.style.\
    format({'%_scraps_byn' : "{:.1%}", '%_scraps_quantity' : "{:.1%}"}, thousands=' ', decimal=',').\
    set_properties(subset=['sales_byn', 'scraps_byn', '%_scraps_byn'], **{'width': '150px'}).\
    set_caption(f"Scraps and losses in {period} by sub brands more 2,5%")

,sales_byn,sales_quantity,scraps_byn,scraps_quantity,%_scraps_byn,%_scraps_quantity
sub_brand,,,,,,
PR M 900,1 146 174,473 710,106 980,44 210,"9,3%","9,3%"
AB 1110,899 390,1 045 810,49 125,58 440,"5,5%","5,6%"
AB 260 N,8 408 450,4 941 740,352 043,214 460,"4,2%","4,3%"
AB 220,457 737,286 940,17 166,10 820,"3,8%","3,8%"
AB 130 N,1 653 410,1 483 600,59 165,52 420,"3,6%","3,5%"
RS CR 45,1 295 931,915 100,46 357,24 050,"3,6%","2,6%"
RS 90,1 592 455,1 097 480,45 444,26 200,"2,9%","2,4%"
AB 140,3 132 233,2 519 750,82 455,65 760,"2,6%","2,6%"


Высокий уровень PR M 900 обусловлен тем, что в конце сентября был осуществлен вывод продукта из продажи.  

Высокий уровень у RS CR 45 и RS 90 может быть обусловлен тем, что мы не угадали с влиянием понижения цены на спрос и/или с влиянием изменения изображения на баночках на спрос (продукт детского питания, изображения из известных детских мультиков). Также осенью всегда изменяется уровень сезонности на данный типа продукта. Возможно требуется пересмотреть расчет уровня сезонности.


In [41]:
df_sr_customers = pd.merge(df_sales_returns, df_customers, how = 'left', on = ['ship_to'])

In [42]:
list_customers_nan = list(df_sr_customers['ship_to'][df_sr_customers['client'].isnull()].value_counts())
list_customers_nan

[]

In [43]:
if not list_customers_nan:
    print('there is all information about the clients')
else:
    raise UserWarning('UPDATE LIST OF CUSTOMERS')

there is all information about the clients


In [44]:
#df_sr_customers

In [45]:
df_clients = pd.pivot_table(df_sr_customers, index = ['client'],
                        values = ['sales_quantity', 'sales_byn', 'scraps_quantity', 'scraps_byn'],
                        aggfunc = 'sum')

df_clients['%_scraps_byn'] = df_clients['scraps_byn'] / df_clients['sales_byn']

df_clients['%_scraps_quantity'] = df_clients['scraps_quantity'] / df_clients['sales_quantity']

df_clients = df_clients.astype({'sales_byn': 'int64', 'scraps_byn' : 'int64'})

#df_clients

In [46]:
df_clients.style.\
    map(color_, subset=['%_scraps_byn','%_scraps_quantity']).\
    format({'%_scraps_byn' : "{:.1%}", '%_scraps_quantity' : "{:.1%}"}, thousands=' ', decimal=',').\
    set_properties(subset=['sales_byn', 'scraps_byn', '%_scraps_byn'], **{'width': '150px'}).\
    set_caption(f"Scraps and losses in {period} by clients")

,sales_byn,sales_quantity,scraps_byn,scraps_quantity,%_scraps_byn,%_scraps_quantity
client,,,,,,
Alm,1 642 958,1 295 960,0,0,"0,0%","0,0%"
Bel,1 648 069,1 215 170,58 703,42 960,"3,6%","3,5%"
Dob,3 692 153,3 238 930,0,0,"0,0%","0,0%"
E-d,946 720,588 130,7 464,5 690,"0,8%","1,0%"
Eur,10 074 055,8 222 860,296 022,207 720,"2,9%","2,5%"
GRI,3 388 713,2 459 530,42 861,33 160,"1,3%","1,3%"
Gro,4 172 144,4 060 390,31 414,26 920,"0,8%","0,7%"
Hip,3 305 817,2 507 620,48 642,35 500,"1,5%","1,4%"
Hit,2 421 246,2 300 500,85 422,57 210,"3,5%","2,5%"


Высокий уровень возвратов в торговых сетях может быть обусловлен ценовой политикой данных сетей или уровнем ответственности торговых представителей, которые работают в этих сетях.

In [53]:
df_sales_people_sku = pd.merge(df_sr_customers, df_sku, how='left', on=['material'])

In [54]:
df_sales_people_sku

,ship_to,material,sales_quantity,sales_byn,scraps_quantity,scraps_byn,cluster,client,sales_people_name,region,brand,sub_brand,sku_name,active_status
0,850232377,48949,480,456.0,0,0.0,Cluster 2,Oth,Станислав Ш.,Minsk,DM,DM 105,DM-DM 105-48949,1
1,850232377,97192,180,158.4,0,0.0,Cluster 2,Oth,Станислав Ш.,Minsk,DM,DM 105,DM-DM 105-97192,1
2,850232377,99639,320,483.2,0,0.0,Cluster 2,Oth,Станислав Ш.,Minsk,DM,DM 130,DM-DM 130-99639,1
3,850232377,99640,400,604.0,0,0.0,Cluster 2,Oth,Станислав Ш.,Minsk,DM,DM 130,DM-DM 130-99640,1
4,850232377,101209,120,169.2,10,19.4,Cluster 2,Oth,Станислав Ш.,Minsk,RS,RS CR 45,RS-RS CR 45-101209,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106833,850310862,194717,70,119.0,0,0.0,Cluster 0,Oth,Елизавета Г.,Minsk region-2,AB,AB 260 N,AB-AB 260 N-194717,1
106834,850310862,194729,130,221.0,0,0.0,Cluster 0,Oth,Елизавета Г.,Minsk region-2,AB,AB 260 N,AB-AB 260 N-194729,1
106835,850310862,194858,60,77.4,0,0.0,Cluster 0,Oth,Елизавета Г.,Minsk region-2,AB,AB 140,AB-AB 140-194858,1
106836,850310862,196459,60,67.2,0,0.0,Cluster 0,Oth,Елизавета Г.,Minsk region-2,ACM,ACM 1,ACM-ACM 1-196459,1


In [47]:
df_sales_people = pd.pivot_table(df_sr_customers, index = ['sales_people_name'],
                        values = ['sales_quantity', 'sales_byn', 'scraps_quantity', 'scraps_byn'],
                        aggfunc = 'sum')

df_sales_people['%_scraps_byn'] = df_sales_people['scraps_byn'] / df_sales_people['sales_byn']

df_sales_people['%_scraps_quantity'] = df_sales_people['scraps_quantity'] / df_sales_people['sales_quantity']

df_sales_people = df_sales_people.astype({'sales_byn': 'int64', 'scraps_byn' : 'int64'})

df_sales_people = df_sales_people.sort_values('%_scraps_byn', ascending=False)

#df_sales_people

In [48]:
df_sales_people_1 = df_sales_people[(df_sales_people['%_scraps_byn'] > 0.03) & (df_sales_people['%_scraps_byn'] <= 1)]

#df_sales_people_1

In [49]:
df_sales_people.style.\
    map(color_, subset=['%_scraps_byn','%_scraps_quantity']).\
    format({'%_scraps_byn' : "{:.1%}", '%_scraps_quantity' : "{:.1%}"}, thousands=' ', decimal=',').\
    set_properties(subset=['sales_byn', 'scraps_byn', '%_scraps_byn'], **{'width': '150px'}).\
    set_caption(f"Scraps and losses in {period} by sales people name")

,sales_byn,sales_quantity,scraps_byn,scraps_quantity,%_scraps_byn,%_scraps_quantity
sales_people_name,,,,,,
Татьяна Д.,542 966,424 910,50 446,32 230,"9,3%","7,6%"
Анастасия Ф.,274 919,208 360,15 941,12 970,"5,8%","6,2%"
Наталья Л.,644 426,530 830,27 378,19 680,"4,2%","3,7%"
Наталья Г.,716 084,597 530,27 733,19 250,"3,9%","3,2%"
Юлия М.,673 657,559 220,26 007,19 280,"3,9%","3,4%"
Татьяна К.,815 454,659 410,29 007,20 880,"3,6%","3,2%"
Лилия К.,300 601,231 050,10 641,8 420,"3,5%","3,6%"
Павел Т.,673 632,562 080,23 636,16 340,"3,5%","2,9%"
Марина Ж.,451 060,362 310,15 800,11 470,"3,5%","3,2%"


In [50]:
df_sales_people_1.style.\
    map(color_, subset=['%_scraps_byn','%_scraps_quantity']).\
    format({'%_scraps_byn' : "{:.1%}", '%_scraps_quantity' : "{:.1%}"}, thousands=' ', decimal=',').\
    set_properties(subset=['sales_byn', 'scraps_byn', '%_scraps_byn'], **{'width': '150px'}).\
    set_caption(f"Scraps and losses in {period} by sales people name more 3%")

,sales_byn,sales_quantity,scraps_byn,scraps_quantity,%_scraps_byn,%_scraps_quantity
sales_people_name,,,,,,
Татьяна Д.,542 966,424 910,50 446,32 230,"9,3%","7,6%"
Анастасия Ф.,274 919,208 360,15 941,12 970,"5,8%","6,2%"
Наталья Л.,644 426,530 830,27 378,19 680,"4,2%","3,7%"
Наталья Г.,716 084,597 530,27 733,19 250,"3,9%","3,2%"
Юлия М.,673 657,559 220,26 007,19 280,"3,9%","3,4%"
Татьяна К.,815 454,659 410,29 007,20 880,"3,6%","3,2%"
Лилия К.,300 601,231 050,10 641,8 420,"3,5%","3,6%"
Павел Т.,673 632,562 080,23 636,16 340,"3,5%","2,9%"
Марина Ж.,451 060,362 310,15 800,11 470,"3,5%","3,2%"


Такие высокие уровни возвратов требуют детального изучения по каждому сотруднику.


In [61]:
df_sales_people_sku_111 = pd.pivot_table(df_sales_people_sku, index = ['sales_people_name', 'sub_brand'],
                        values = ['sales_quantity', 'sales_byn', 'scraps_quantity', 'scraps_byn'],
                        aggfunc = 'sum')

df_sales_people_sku_111['%_scraps_byn'] = df_sales_people_sku_111['scraps_byn'] / df_sales_people_sku_111['sales_byn']

df_sales_people_sku_111['%_scraps_quantity'] = df_sales_people_sku_111['scraps_quantity'] / df_sales_people_sku_111['sales_quantity']

df_sales_people_sku_111 = df_sales_people_sku_111.astype({'sales_byn': 'int64', 'scraps_byn' : 'int64'})

df_sales_people_sku_111 = df_sales_people_sku_111.sort_values('%_scraps_byn', ascending=False)

(df_sales_people_sku_111.loc['Татьяна Д.', :]).sort_values('%_scraps_byn', ascending=False).style.\
    map(color_, subset=['%_scraps_byn','%_scraps_quantity']).\
    format({'%_scraps_byn' : "{:.1%}", '%_scraps_quantity' : "{:.1%}"}, thousands=' ', decimal=',').\
    set_properties(subset=['sales_byn', 'scraps_byn', '%_scraps_byn'], **{'width': '150px'}).\
    set_caption(f"Scraps and losses in {period} by sales people name (Татьяна Д.)")

,sales_byn,sales_quantity,scraps_byn,scraps_quantity,%_scraps_byn,%_scraps_quantity
sub_brand,,,,,,
AV SP 140,0,0,114,110,inf%,inf%
AM 1 95,0,0,3 565,3 360,inf%,inf%
AV DR 260,0,0,761,550,inf%,inf%
AV DR 260 NS,0,0,544,320,inf%,inf%
AV SP 110,0,0,49,70,inf%,inf%
AV SP 130,0,0,239,280,inf%,inf%
AV SP 130 NS,0,0,111,110,inf%,inf%
DM 190,124,60,82,40,"66,7%","66,7%"
PR M 900,26 335,10 860,6 379,2 630,"24,2%","24,2%"


Разберем работу конкретного сотрудника.  

Несмотря на то, что продукт PR M 900 был недавно выведен из продажи, такой высокий уровень возвратов не может быть обусловлен выводом или непопаданием в прогноз. Возможно, торговый представитель делал заказ наугад.  

Высокий уровень возвратов у продукта PR B 160 нельзя объяснить ничем, кроме как ошибки сотрудника: у данного продукта большой срок годности и стабильный уровень спроса. Так как данный продукт является одним из самых дорогих, то, возможно, сотрудник хотел выполнить план по продажам и перезаказал продукт (один из самых дорогих продуктов + плюс большой срок годности, однако продукта оказалось слишком много). 


### Создание базы данных:

Использование postgres и dbeaver  
Построение модели и установка ограничений  
Загрузка данных



### Создание dashboard в Power BI



<img src='dashboard.png'>